In [30]:
import requests
import pandas as pd
from dotenv import load_dotenv
import time
import folium
import os
load_dotenv()
KAKAO_REST_API_KEY = os.getenv("KAKAO_REST_API_KEY")

def kakao_geocode(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['documents']:
            latitude = data['documents'][0]['y']
            longitude = data['documents'][0]['x']
            print(f"Address: {address}, Latitude: {latitude}, Longitude: {longitude}")
            return latitude, longitude
        else:
            print("No results found.")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None

In [31]:
gyeongnam_offices = pd.read_csv('../data/경남_관공서_현황(2023).csv', encoding='utf-8')
gyeongnam_offices

,시군구,행정복지센터명,우편번호,도로명주소
0,창원시 의창구,동읍행정복지센터,51128,경상남도 창원시 의창구 동읍 동읍로 88
1,창원시 의창구,북면행정복지센터,51103,경상남도 창원시 의창구 북면 천주로 1085
2,창원시 의창구,대산면행정복지센터,51124,경상남도 창원시 의창구 대산면 가술산단동로 10
3,창원시 의창구,의창동행정복지센터,51194,경상남도 창원시 의창구 서상로12번길 75
4,창원시 의창구,팔룡동행정복지센터,51374,경상남도 창원시 의창구 팔용로 435
...,...,...,...,...
300,합천군,쌍백면사무소,50218,경상남도 합천군 쌍백면 쌍백중앙로 63
301,합천군,삼가면사무소,50222,경상남도 합천군 삼가면 삼가중앙2길 12-8
302,합천군,가회면사무소,50226,경상남도 합천군 가회면 황매산로 52
303,합천군,대병면사무소,50216,경상남도 합천군 대병면 신성동길 23


In [33]:

lats, lons = [], []

for addr in gyeongnam_offices['도로명주소']:
    try:
        result = kakao_geocode(addr)
        if result is not None:
            lat, lon = result
        else:
            # 주소를 못 찾은 경우 패스
            lat, lon = None, None
    except:
        # 예외 발생 시 패스
        lat, lon = None, None

    lats.append(lat)
    lons.append(lon)
    time.sleep(0.2)  # API 제한 대응
# 위경도 컬럼 추가
gyeongnam_offices['lat'] = lats
gyeongnam_offices['lon'] = lons

m = folium.Map(location=[35.23, 128.68], zoom_start=9)

for i, row in gyeongnam_offices.dropna(subset=['lat', 'lon']).iterrows():
    folium.Marker(
        [row['lat'], row['lon']],
        popup=row['행정복지센터명'],
        tooltip=row['시군구'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# 결과 저장
m.save("경남_행정복지센터_지도.html")

Address: 경상남도 창원시 의창구 동읍 동읍로 88, Latitude: 35.2843693071283, Longitude: 128.68272614556
Address: 경상남도 창원시 의창구 북면 천주로 1085, Latitude: 35.3479738689755, Longitude: 128.607312660548
Address: 경상남도 창원시 의창구 대산면 가술산단동로 10, Latitude: 35.3310298079288, Longitude: 128.708706806491
Address: 경상남도 창원시 의창구 서상로12번길 75, Latitude: 35.2620484294053, Longitude: 128.622881591564
Address: 경상남도 창원시 의창구 팔용로 435, Latitude: 35.2554790726021, Longitude: 128.612831987917
Address: 경상남도 창원시 의창구 태복산로15번길 8, Latitude: 35.2539403841184, Longitude: 128.63992461167
Address: 경상남도 창원시 의창구 대봉로26번길 5, Latitude: 35.2519212278231, Longitude: 128.667586778088
Address: 경상남도 창원시 성산구 원이대로473번길 19-14, Latitude: 35.2358298524995, Longitude: 128.669543020891
Address: 경상남도 창원시 성산구 외동반림로 5, Latitude: 35.221054241716, Longitude: 128.67370692322
No results found.
Address: 경상남도 창원시 성산구 동산로 73, Latitude: 35.2159921257208, Longitude: 128.687360480742
Address: 경상남도 창원시 성산구 대암로 109, Latitude: 35.2159004913944, Longitude: 128.708705534947
Ad

In [34]:
from folium.plugins import HeatMap

# 지도 생성
m = folium.Map(location=[35.23, 128.68], zoom_start=9)

# 위경도 좌표만 추출 (결측치 제거)
heat_data = gyeongnam_offices.dropna(subset=['lat', 'lon'])[['lat', 'lon']].values.tolist()

# 히트맵 추가
HeatMap(heat_data, radius=15, blur=10, max_zoom=12).add_to(m)

# 저장 및 표시
m.save("경남_행정복지센터_히트맵.html")